Goal: Cluster heros using player-level features.

# Introduction to Data Science: Exploratory Data Analysis

### Data Sources

- open dota api: https://docs.opendota.com/#section/Introduction
- open dota github: https://github.com/odota
- spec.json: contains information / specification for the Open Dota API. https://api.opendota.com/api


### Notes
- Each hero is associated with a unique ID
- "/players/{account_id}/recentMatches" has solid data on recent matches per-player
- "/players/{account_id}/totals" has some solid total data across all matches per-player
- "/heroes" has the mapping from hero id to name (plus roles, etc)
- "/heroStats" has base stats of all heroes (health, agi gain, etc)

### TODO
- Generate mapping function from hero id to hero name

In [14]:
import json

import requests
import pandas as pd
pd.set_option('display.max_rows', 500)

## Get basic information

In [21]:
# load spec
with open("spec.json") as f:
    spec = json.load(f)

spec.keys()

dict_keys(['swagger', 'info', 'securityDefinitions', 'host', 'basePath', 'produces', 'paths'])

## Messy exploration

In [27]:
# basic testing

me = "37244929"  # my account ID
req = f"https://api.opendota.com/api/players/{me}"
r = requests.get(req)

me = r.json()  # got info about myself!
me

{'tracked_until': None,
 'solo_competitive_rank': 3957,
 'profile': {'account_id': 37244929,
  'personaname': 'Lofty',
  'name': None,
  'plus': False,
  'cheese': 0,
  'steamid': '76561197997510657',
  'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/53/5303c623b9f11b3528cdac1ad8e15ec9425a07c8.jpg',
  'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/53/5303c623b9f11b3528cdac1ad8e15ec9425a07c8_medium.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/53/5303c623b9f11b3528cdac1ad8e15ec9425a07c8_full.jpg',
  'profileurl': 'https://steamcommunity.com/id/Noak3/',
  'last_login': None,
  'loccountrycode': None,
  'is_contributor': False},
 'competitive_rank': None,
 'mmr_estimate': {'estimate': 3387},
 'rank_tier': 53,
 'leaderboard_rank': None}

In [47]:
def grab_json(req):
    """
    Grab info specified in `spec.json`.
    
    req: str
    Returns: dict
    """
    api = "https://api.opendota.com/api"
    req = api + req
    r = requests.get(req)
    return r.json()

grab_json("/players/37244929")

me = "/players/37244929"
grab_json(me + "/heroes")[:3]  # Winrates, number of games, etc for all heroes (by hero id)

[{'hero_id': '82',
  'last_played': 1574441970,
  'games': 227,
  'win': 126,
  'with_games': 19,
  'with_win': 6,
  'against_games': 29,
  'against_win': 20},
 {'hero_id': '53',
  'last_played': 1573780716,
  'games': 152,
  'win': 86,
  'with_games': 125,
  'with_win': 55,
  'against_games': 193,
  'against_win': 87},
 {'hero_id': '80',
  'last_played': 1574833141,
  'games': 120,
  'win': 63,
  'with_games': 34,
  'with_win': 13,
  'against_games': 61,
  'against_win': 31}]

In [48]:
# Can use `json_normalize` to make a dataframe when there's nested data
pd.json_normalize(grab_json("/players/37244929"))

,tracked_until,solo_competitive_rank,competitive_rank,rank_tier,leaderboard_rank,mmr_estimate.estimate,profile.account_id,profile.personaname,profile.name,profile.plus,profile.cheese,profile.steamid,profile.avatar,profile.avatarmedium,profile.avatarfull,profile.profileurl,profile.last_login,profile.loccountrycode,profile.is_contributor
0,None,3957,None,53,None,3387,37244929,Lofty,None,False,0,76561197997510657,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcdn-a.akamaihd.net/steamcommunity...,https://steamcommunity.com/id/Noak3/,None,None,False


### Per-Player Info

In [80]:
grab_json(me + "/wl")

{'win': 1005, 'lose': 1044}

In [83]:
grab_json(me + "/recentMatches")[:2]  # stats for recent matches

[{'match_id': 5400511138,
  'player_slot': 1,
  'radiant_win': False,
  'duration': 3179,
  'game_mode': 22,
  'lobby_type': 0,
  'hero_id': 110,
  'start_time': 1588782589,
  'version': 21,
  'kills': 2,
  'deaths': 8,
  'assists': 14,
  'skill': None,
  'xp_per_min': 402,
  'gold_per_min': 208,
  'hero_damage': 21847,
  'tower_damage': 299,
  'hero_healing': 7028,
  'last_hits': 60,
  'lane': 1,
  'lane_role': 1,
  'is_roaming': False,
  'cluster': 123,
  'leaver_status': 0,
  'party_size': 2},
 {'match_id': 5399348019,
  'player_slot': 130,
  'radiant_win': False,
  'duration': 2390,
  'game_mode': 22,
  'lobby_type': 7,
  'hero_id': 90,
  'start_time': 1588731136,
  'version': 21,
  'kills': 1,
  'deaths': 5,
  'assists': 10,
  'skill': 2,
  'xp_per_min': 386,
  'gold_per_min': 219,
  'hero_damage': 9260,
  'tower_damage': 244,
  'hero_healing': 828,
  'last_hits': 37,
  'lane': 3,
  'lane_role': 1,
  'is_roaming': False,
  'cluster': 123,
  'leaver_status': 0,
  'party_size': 3}]

In [85]:
grab_json(me + "/matches")[:2]  # different (worse) stats for all matches

[{'match_id': 5400511138,
  'player_slot': 1,
  'radiant_win': False,
  'duration': 3179,
  'game_mode': 22,
  'lobby_type': 0,
  'hero_id': 110,
  'start_time': 1588782589,
  'version': 21,
  'kills': 2,
  'deaths': 8,
  'assists': 14,
  'skill': None,
  'leaver_status': 0,
  'party_size': 2},
 {'match_id': 5399348019,
  'player_slot': 130,
  'radiant_win': False,
  'duration': 2390,
  'game_mode': 22,
  'lobby_type': 7,
  'hero_id': 90,
  'start_time': 1588731136,
  'version': 21,
  'kills': 1,
  'deaths': 5,
  'assists': 10,
  'skill': 2,
  'leaver_status': 0,
  'party_size': 3}]

In [87]:
grab_json(me + "/heroes")[:2]  # stats per-hero

[{'hero_id': '82',
  'last_played': 1574441970,
  'games': 227,
  'win': 126,
  'with_games': 19,
  'with_win': 6,
  'against_games': 29,
  'against_win': 20},
 {'hero_id': '53',
  'last_played': 1573780716,
  'games': 152,
  'win': 86,
  'with_games': 125,
  'with_win': 55,
  'against_games': 193,
  'against_win': 87}]

In [53]:
grab_json(me + "/peers")[:2]  # who do I play with the most?

[{'account_id': 44647981,
  'last_played': 1440477579,
  'win': 306,
  'games': 556,
  'with_win': 306,
  'with_games': 556,
  'against_win': 0,
  'against_games': 0,
  'with_gpm_sum': 224858,
  'with_xpm_sum': 266751,
  'personaname': 'MuLTiVeRsE',
  'name': None,
  'is_contributor': False,
  'last_login': None,
  'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/61/61690d4fc112968902b2b61139a3b2c90b626f4c.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/61/61690d4fc112968902b2b61139a3b2c90b626f4c_full.jpg'},
 {'account_id': 80785200,
  'last_played': 1440477579,
  'win': 179,
  'games': 348,
  'with_win': 179,
  'with_games': 348,
  'against_win': 0,
  'against_games': 0,
  'with_gpm_sum': 147113,
  'with_xpm_sum': 173788,
  'personaname': 'shrubbery',
  'name': None,
  'is_contributor': False,
  'last_login': None,
  'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/4f/4f8c27ef869a

In [56]:
grab_json(me + "/pros")[:2]  # have I played with or against pros? (I think?)

[{'account_id': 87177591,
  'name': 'Fear',
  'country_code': 'us',
  'fantasy_role': 2,
  'team_id': 7681441,
  'team_name': 'business associates',
  'team_tag': 'business',
  'is_locked': True,
  'is_pro': True,
  'locked_until': None,
  'steamid': '76561198047443319',
  'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/e0/e01b0b13935bda6b33f375be260945541e785e02.jpg',
  'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/e0/e01b0b13935bda6b33f375be260945541e785e02_medium.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/e0/e01b0b13935bda6b33f375be260945541e785e02_full.jpg',
  'profileurl': 'https://steamcommunity.com/profiles/76561198047443319/',
  'personaname': '2pac',
  'last_login': None,
  'full_history_time': '2020-05-03T23:00:29.119Z',
  'cheese': 0,
  'fh_unavailable': True,
  'loccountrycode': None,
  'last_match_time': '2020-04-30T20:18:09.000Z',
  'plus': None,
  'las

In [49]:
grab_json(me + "/totals")  # total info across all games

[{'field': 'kills', 'n': 2049, 'sum': 14942},
 {'field': 'deaths', 'n': 2049, 'sum': 14660},
 {'field': 'assists', 'n': 2049, 'sum': 22986},
 {'field': 'kda', 'n': 2049, 'sum': 5252},
 {'field': 'gold_per_min', 'n': 2049, 'sum': 812238},
 {'field': 'xp_per_min', 'n': 2049, 'sum': 984327},
 {'field': 'last_hits', 'n': 2049, 'sum': 283712},
 {'field': 'denies', 'n': 2049, 'sum': 14067},
 {'field': 'lane_efficiency_pct', 'n': 100, 'sum': 4913},
 {'field': 'duration', 'n': 2049, 'sum': 4839978},
 {'field': 'level', 'n': 2049, 'sum': 37341},
 {'field': 'hero_damage', 'n': 999, 'sum': 13571391},
 {'field': 'tower_damage', 'n': 999, 'sum': 1597989},
 {'field': 'hero_healing', 'n': 999, 'sum': 413453},
 {'field': 'stuns', 'n': 96, 'sum': 2539.1796498400004},
 {'field': 'tower_kills', 'n': 100, 'sum': 105},
 {'field': 'neutral_kills', 'n': 100, 'sum': 5138},
 {'field': 'courier_kills', 'n': 100, 'sum': 9},
 {'field': 'purchase_tpscroll', 'n': 100, 'sum': 730},
 {'field': 'purchase_ward_observer

In [57]:
grab_json(me + "/counts")  # How many games have I played and won under certain conditions?

{'leaver_status': {'0': {'games': 1919, 'win': 980},
  '1': {'games': 69, 'win': 11},
  '2': {'games': 42, 'win': 11},
  '3': {'games': 10, 'win': 3},
  '4': {'games': 9, 'win': 0}},
 'game_mode': {'0': {'games': 460, 'win': 240},
  '1': {'games': 823, 'win': 418},
  '2': {'games': 42, 'win': 18},
  '3': {'games': 351, 'win': 173},
  '4': {'games': 45, 'win': 18},
  '5': {'games': 49, 'win': 24},
  '12': {'games': 7, 'win': 2},
  '16': {'games': 59, 'win': 28},
  '22': {'games': 213, 'win': 84}},
 'lobby_type': {'0': {'games': 1952, 'win': 964},
  '7': {'games': 97, 'win': 41}},
 'lane_role': {'0': {'games': 1949, 'win': 962},
  '1': {'games': 42, 'win': 18},
  '2': {'games': 24, 'win': 13},
  '3': {'games': 33, 'win': 12},
  '4': {'games': 1, 'win': 0}},
 'region': {'0': {'games': 49, 'win': 26},
  '1': {'games': 1130, 'win': 582},
  '2': {'games': 853, 'win': 392},
  '3': {'games': 17, 'win': 5}},
 'patch': {'2': {'games': 112, 'win': 58},
  '3': {'games': 92, 'win': 49},
  '4': {'ga

In [58]:
grab_json(me + "/wardmap")  # Ward placement locations. Don't understand value correspondence to placement -- probably a coordinate grid?

{'obs': {'76': {'114': 1},
  '78': {'152': 1},
  '80': {'130': 1, '152': 1, '158': 1},
  '82': {'128': 1, '152': 1, '154': 1, '156': 1},
  '84': {'76': 1, '144': 1, '152': 3, '154': 1},
  '86': {'100': 1,
   '114': 1,
   '142': 1,
   '144': 2,
   '146': 2,
   '152': 1,
   '162': 3,
   '164': 3},
  '88': {'92': 1, '142': 2, '152': 1},
  '90': {'132': 1},
  '92': {'96': 1, '110': 1, '116': 1, '120': 2, '132': 1, '166': 1},
  '94': {'112': 2, '118': 2, '120': 1, '142': 1},
  '96': {'80': 1, '142': 1},
  '98': {'78': 1, '90': 1, '92': 2, '132': 1, '174': 1},
  '100': {'72': 1,
   '114': 5,
   '118': 1,
   '120': 1,
   '126': 1,
   '132': 3,
   '136': 1,
   '152': 3,
   '154': 1,
   '166': 1,
   '174': 2},
  '102': {'116': 1},
  '104': {'126': 1, '128': 1, '130': 1},
  '106': {'128': 1, '152': 3},
  '108': {'80': 1, '110': 1, '122': 1, '126': 1, '128': 1, '152': 1, '154': 1},
  '110': {'78': 1, '126': 1, '148': 2, '156': 2, '164': 1},
  '112': {'88': 1, '90': 1, '130': 3, '152': 1, '156': 1

In [76]:
pd.Series(grab_json(me + "/wordcloud")['my_word_counts']).sort_values(ascending=False)[:20]  # Wordcloud of things I've said in chat

gg      57
you     43
a       31
i       27
we      24
that    22
the     21
game    20
it      18
for     18
this    18
was     17
and     17
just    16
lol     16
to      16
so      15
my      15
is      15
bro     15
dtype: int64

In [77]:
grab_json(me + "/ratings")  # competitive ratings

[{'account_id': 37244929,
  'match_id': None,
  'solo_competitive_rank': 3957,
  'competitive_rank': None,
  'time': '2015-11-16T22:00:44.986Z'}]

In [78]:
grab_json(me + "/rankings")  # rank by hero

[{'hero_id': 92,
  'score': 4273.40284553964,
  'percent_rank': 0.934038267875126,
  'card': 993000},
 {'hero_id': 51,
  'score': 4512.90790232725,
  'percent_rank': 0.924909747292419,
  'card': 2770000},
 {'hero_id': 99,
  'score': 4405.53526435902,
  'percent_rank': 0.909598511034299,
  'card': 3761000},
 {'hero_id': 83,
  'score': 4353.88184525001,
  'percent_rank': 0.894691224268689,
  'card': 2307500},
 {'hero_id': 5,
  'score': 4362.03025871363,
  'percent_rank': 0.847563515201999,
  'card': 3601500},
 {'hero_id': 90,
  'score': 4176.97166980042,
  'percent_rank': 0.824808699154249,
  'card': 2483000},
 {'hero_id': 101,
  'score': 4260.7454065886,
  'percent_rank': 0.811287477954145,
  'card': 3402000},
 {'hero_id': 2,
  'score': 4275.48412765182,
  'percent_rank': 0.809979838709677,
  'card': 3968000},
 {'hero_id': 73,
  'score': 4168.37836181533,
  'percent_rank': 0.786553406831737,
  'card': 2766500},
 {'hero_id': 59,
  'score': 4132.37496193788,
  'percent_rank': 0.7851720239

In [90]:
grab_json("/proPlayers")[:2]  # pro player info

[{'account_id': 88470,
  'steamid': '76561197960354198',
  'avatar': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/b9/b92793127bfa6ceb1edbd2b7b25011b1dc6db89e.jpg',
  'avatarmedium': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/b9/b92793127bfa6ceb1edbd2b7b25011b1dc6db89e_medium.jpg',
  'avatarfull': 'https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/b9/b92793127bfa6ceb1edbd2b7b25011b1dc6db89e_full.jpg',
  'profileurl': 'https://steamcommunity.com/id/misterdurst69/',
  'personaname': 'Tzy丶',
  'last_login': None,
  'full_history_time': '2020-05-06T05:02:30.803Z',
  'cheese': 0,
  'fh_unavailable': False,
  'loccountrycode': 'CN',
  'last_match_time': '2020-05-06T16:21:44.000Z',
  'plus': True,
  'name': 'TZY',
  'country_code': 'cn',
  'fantasy_role': 1,
  'team_id': 6020739,
  'team_name': '',
  'team_tag': '',
  'is_locked': False,
  'is_pro': True,
  'locked_until': None},
 {'account_id': 1296625,
  'steamid': '76561

In [100]:
distributions = grab_json("/distributions")  # MMR data by bracket and country
distributions.keys()

dict_keys(['ranks', 'mmr', 'country_mmr'])

### Hero Information

In [111]:
hero_data = grab_json("/heroes")
pd.DataFrame(hero_data).set_index("id", drop=True)  # useful: heroes, along with basic info

,name,localized_name,primary_attr,attack_type,roles,legs
id,,,,,,
1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler]",2
3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",4
4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Jungler, Nuker, Initiator]",2
5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker, Jungler]",2
6,npc_dota_hero_drow_ranger,Drow Ranger,agi,Ranged,"[Carry, Disabler, Pusher]",2
7,npc_dota_hero_earthshaker,Earthshaker,str,Melee,"[Support, Initiator, Disabler, Nuker]",2
8,npc_dota_hero_juggernaut,Juggernaut,agi,Melee,"[Carry, Pusher, Escape]",2
9,npc_dota_hero_mirana,Mirana,agi,Ranged,"[Carry, Support, Escape, Nuker, Disabler]",2


In [114]:
# Using Anti-Mage as a test hero
grab_json("/heroes/1/matches")[:2]  # matches with that hero

[{'match_id': 5400070079,
  'start_time': 1588769130,
  'duration': 2746,
  'radiant_win': False,
  'leagueid': 11099,
  'league_name': '中国DOTA2发展联赛',
  'radiant': True,
  'player_slot': 128,
  'account_id': 136177710,
  'kills': 13,
  'deaths': 0,
  'assists': 9},
 {'match_id': 5400050652,
  'start_time': 1588768376,
  'duration': 1904,
  'radiant_win': False,
  'leagueid': 11966,
  'league_name': 'Vulcan Champion League',
  'radiant': False,
  'player_slot': 2,
  'account_id': 166769877,
  'kills': 2,
  'deaths': 2,
  'assists': 0}]

In [122]:
grab_json("/heroStats")[0]

{'id': 1,
 'name': 'npc_dota_hero_antimage',
 'localized_name': 'Anti-Mage',
 'primary_attr': 'agi',
 'attack_type': 'Melee',
 'roles': ['Carry', 'Escape', 'Nuker'],
 'img': '/apps/dota2/images/heroes/antimage_full.png?',
 'icon': '/apps/dota2/images/heroes/antimage_icon.png',
 'base_health': 200,
 'base_health_regen': 0.25,
 'base_mana': 75,
 'base_mana_regen': 0,
 'base_armor': -1,
 'base_mr': 25,
 'base_attack_min': 29,
 'base_attack_max': 33,
 'base_str': 23,
 'base_agi': 24,
 'base_int': 12,
 'str_gain': 1.3,
 'agi_gain': 3,
 'int_gain': 1.8,
 'attack_range': 150,
 'projectile_speed': 0,
 'attack_rate': 1.4,
 'move_speed': 310,
 'turn_rate': 0.5,
 'cm_enabled': True,
 'legs': 2,
 'pro_ban': 128,
 'hero_id': 1,
 'pro_win': 61,
 'pro_pick': 107,
 '1_pick': 15204,
 '1_win': 7714,
 '2_pick': 43096,
 '2_win': 22148,
 '3_pick': 66959,
 '3_win': 34868,
 '4_pick': 71889,
 '4_win': 37602,
 '5_pick': 47889,
 '5_win': 25203,
 '6_pick': 19996,
 '6_win': 10196,
 '7_pick': 6790,
 '7_win': 3480,

In [116]:
grab_json("/heroes/1/matchups")[:5]  # winrates vs other heroes, but numbers are too small to be totals, so maybe a representative subset?

[{'hero_id': 16, 'games_played': 176, 'wins': 87},
 {'hero_id': 52, 'games_played': 125, 'wins': 60},
 {'hero_id': 121, 'games_played': 117, 'wins': 68},
 {'hero_id': 106, 'games_played': 115, 'wins': 61},
 {'hero_id': 10, 'games_played': 106, 'wins': 55}]

In [117]:
grab_json("/heroes/1/durations")

[{'duration_bin': 5700, 'games_played': 1, 'wins': 0},
 {'duration_bin': 3300, 'games_played': 182, 'wins': 95},
 {'duration_bin': 7500, 'games_played': 1, 'wins': 0},
 {'duration_bin': 1800, 'games_played': 1034, 'wins': 530},
 {'duration_bin': 3900, 'games_played': 34, 'wins': 15},
 {'duration_bin': 300, 'games_played': 12, 'wins': 6},
 {'duration_bin': 7200, 'games_played': 1, 'wins': 1},
 {'duration_bin': 3000, 'games_played': 273, 'wins': 157},
 {'duration_bin': 4200, 'games_played': 20, 'wins': 9},
 {'duration_bin': 4500, 'games_played': 15, 'wins': 9},
 {'duration_bin': 1200, 'games_played': 604, 'wins': 207},
 {'duration_bin': 2700, 'games_played': 445, 'wins': 269},
 {'duration_bin': 5100, 'games_played': 2, 'wins': 0},
 {'duration_bin': 2100, 'games_played': 839, 'wins': 484},
 {'duration_bin': 5400, 'games_played': 2, 'wins': 2},
 {'duration_bin': 900, 'games_played': 246, 'wins': 90},
 {'duration_bin': 12000, 'games_played': 1, 'wins': 0},
 {'duration_bin': 600, 'games_play

In [118]:
grab_json("/heroes/1/itemPopularity")

{'start_game_items': {'11': 92,
  '12': 9,
  '14': 35,
  '16': 186,
  '20': 23,
  '27': 10,
  '34': 10,
  '39': 30,
  '42': 5,
  '43': 2,
  '44': 100,
  '94': 1,
  '181': 14,
  '237': 6,
  '241': 12},
 'early_game_items': {'3': 21,
  '8': 2,
  '26': 1,
  '56': 79,
  '57': 3,
  '63': 50,
  '69': 2},
 'mid_game_items': {'24': 80,
  '52': 1,
  '53': 1,
  '54': 1,
  '81': 2,
  '114': 1,
  '137': 1,
  '139': 1,
  '143': 28,
  '145': 92,
  '147': 67,
  '160': 1,
  '168': 1,
  '170': 82},
 'late_game_items': {'108': 1,
  '112': 6,
  '116': 16,
  '123': 2,
  '133': 7,
  '135': 1,
  '139': 29,
  '147': 12,
  '154': 1,
  '156': 1,
  '160': 23,
  '176': 2,
  '208': 48,
  '249': 1,
  '271': 1}}